### Creating the project

In [ ]:
# create the project:
$ django-admin startproject mysite

# the infrastructure:
mysite/ # name doesn't matter
    manage.py # for  interaction with project
    mysite/
        __init__.py
        settings.py # project settings
        urls.py # "table of content"
        wsgi.py # start for wsgi servers

In [ ]:
# in folder mysite run the server
$ python manage.py runserver 8000
# access it at http://127.0.0.1:8000/

### Creating an app

In [ ]:
# in main folder mysite/ create an app
$ python manage.py startapp polls

# the infrastructure
polls/
    __init__.py
    admin.py
    apps.py
    migrations/
        __init__.py
    models.py
    tests.py
    views.py
    -> urls.py # add later

#### Write the first view

In [ ]:
# edit polls/views.py
from django.http import HttpResponse


def index(request):
    return HttpResponse("Hello, world. You're at the polls index.")
# The simplest view in Django.
# To call it we need to map it to URL.

In [ ]:
# Add "urls.py"  to the polls/ folder
# edit polls/urls.py
# Show, what url needs to be accessed to get to the view.
from django.conf.urls import url

from . import views

urlpatterns = [
    url(r'^$', views.index, name='index'),
]

In [ ]:
# Point the root URLconf at polls.urls module
# edit mysite/urls.py
from django.conf.urls import include, url
from django.contrib import admin

urlpatterns = [
    url(r'^polls/', include('polls.urls')),
    url(r'^admin/', admin.site.urls),
]

# Always use include for other URL patterns.
# admin.site.urls is the only exception.

In [ ]:
# Try if the view works.
$ python manage.py runserver 8000
# Access it at http://127.0.0.1:8000/polls/
# You should see the text.

#### Database setup

In [ ]:
# change mysite/settings.py
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql',
        'NAME': 'mydatabase',
        'USER': 'mydatabaseuser',
        'PASSWORD': 'mypassword',
        'HOST': '127.0.0.1',
        'PORT': '5432',
    }
}

In [ ]:
# The INSTALLED_APS contains the default apps.
# Some of them need at least one database table, so create it:
$ python manage.py migrate
# It looks at the INSTALLED_APS  and creates neccessaty DBs.
# Check with \dt in postgreSQL.

### Creating models
Essentially the database layout.

In [ ]:
# edit polls/models.py
# Class represents a database table.
# Class attributes represent a table field.
from django.db import models


class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField('date published')
    
    def __str__(self): # The name shown in database/admin page.
        return self.question_text


class Choice(models.Model):
    question = models.ForeignKey(Question, on_delete=models.CASCADE)
    choice_text = models.CharField(max_length=200)
    votes = models.IntegerField(default=0)

#### Activating model

In [ ]:
# Add the new model myseite/settings.py
INSTALLED_APPS = [
    'polls.apps.PollsConfig',
    ...
]

# Tell django you made changes
$ python manage.py makemigrations polls
# Migrations is how django stores changes to models
# What would that migrate do?
$ python manage.py sqlmigrate polls 0001

# Now run migrate again to create those model tables in DB
$ python manage.py migrate
# It takes all the migrations that you haven't applies and runs them.

In [ ]:
1. # Change your models (in models.py).
2. $ python manage.py makemigrations # to create migrations for those changes
3. $ python manage.py migrate # to apply those changes to the database.

### Reset database

In [ ]:
$ python manage.py migrate --fake appname zero
$ python manage.py migrate appname

### Playing with API

In [ ]:
# Enter with
$ python manage.py shell

In [ ]:
# Play with the API
from polls.models import Question, Choice   # Import the model classes we just wrote.

Question.objects.all() # List all Question instances

# Create a question (a row in a table)
from django.utils import timezone
q = Question(question_text="What's new?", pub_date=timezone.now())
q.save() # Save into database!

# Call its attributes
q.id
q.question_text
# Change values
q.question_text = "What's up?"
q.save()

### Create an admin user and admin page

In [ ]:
# Create admin user.
$ python manage.py createsuperuser

In [ ]:
# Enter the admin site at http://127.0.0.1:8000/admin/
$ python manage.py runserver 8000

In [ ]:
# Make the poll app modifiable in admin.
# edit polls/admin.py
from django.contrib import admin

from .models import Question

admin.site.register(Question)

# Now you have the options to add, remove and edit questions.

## Adding more views
The public interface!
In our poll application, we’ll have the following four views:

- Question “index” page – displays the latest few questions.
- Question “detail” page – displays a question text, with no results but with a form to vote.
- Question “results” page – displays results for a particular question.
- Vote action – handles voting for a particular choice in a particular question.

In Django, web pages and other content are delivered by views. Each view is represented by a simple Python function (or method, in the case of class-based views). Django will choose a view by examining the URL that’s requested (to be precise, the part of the URL after the domain name).

In [ ]:
# Edit polls/views.py, add
def detail(request, question_id):
    return HttpResponse("You're looking at question %s." % question_id)

def results(request, question_id):
    response = "You're looking at the results of question %s."
    return HttpResponse(response % question_id)

def vote(request, question_id):
    return HttpResponse("You're voting on question %s." % question_id)

In [ ]:
# Edit polls/urls.py
# Link views to urls.
urlpatterns = [
    # ex: /polls/
    url(r'^$', views.index, name='index'),
    # ex: /polls/5/
    url(r'^(?P<question_id>[0-9]+)/$', views.detail, name='detail'),
    # ex: /polls/5/results/
    url(r'^(?P<question_id>[0-9]+)/results/$', views.results, name='results'),
    # ex: /polls/5/vote/
    url(r'^(?P<question_id>[0-9]+)/vote/$', views.vote, name='vote'),
]

### Views that actually do something

In [ ]:
# Edit polls/views.py
# Use the Django's own API.
# List the latest 5 questions, separated by ",".
def index(request):
    latest_question_list = Question.objects.order_by('-pub_date')[:5]
    output = ', '.join([q.question_text for q in latest_question_list])
    return HttpResponse(output)

There’s a problem here, though: the page’s design is hard-coded in the view. If you want to change the way the page looks, you’ll have to edit this Python code. So let’s use Django’s template system to separate the design from Python by creating a template that the view can use.

### Templates

In [ ]:
# Create index.html in polls/templates/polls/
# You can refer to it simply with polls/index.html
# Variables are passed with the views.py functions (context maps variables to python objects)
{% if latest_question_list %}
    <ul>
    {% for question in latest_question_list %}
        <li><a href="/polls/{{ question.id }}/">{{ question.question_text }}</a></li>
    {% endfor %}
    </ul>
{% else %}
    <p>No polls are available.</p>
{% endif %}

In [ ]:
# Edit polls/views.py
from django.http import HttpResponse
from django.template import loader

from .models import Question


def index(request):
    latest_question_list = Question.objects.order_by('-pub_date')[:5]
    template = loader.get_template('polls/index.html') # Load the template
    context = {'latest_question_list': latest_question_list,} # Map template variables to python objects
    return HttpResponse(template.render(context, request))

In [ ]:
# Second option:
# The render() function takes the request object as its first argument,
# a template name as its second argument and a dictionary as its optional third argument.
# It returns an HttpResponse object of the given template rendered with the given context.
from django.shortcuts import render

from .models import Question

def index(request):
    latest_question_list = Question.objects.order_by('-pub_date')[:5]
    context = {'latest_question_list': latest_question_list}
    return render(request, 'polls/index.html', context)

### Raising a 404 error

In [ ]:
# Edit polls/views.py
# The new concept here: The view raises the Http404 exception if a question with the requested ID doesn’t exist.
from django.http import Http404
from django.shortcuts import render

def detail(request, question_id):
    try:
        question = Question.objects.get(pk=question_id)
    except Question.DoesNotExist:
        raise Http404("Question does not exist")
    return render(request, 'polls/detail.html', {'question': question})

In [ ]:
# Second option:
# The get_object_or_404() function takes a Django model as its first argument
# and an arbitrary number of keyword arguments, which it passes to the
# get() function of the model’s manager. It raises Http404 if the object doesn’t exist.
from django.shortcuts import get_object_or_404, render

from .models import Question
# ...
def detail(request, question_id):
    question = get_object_or_404(Question, pk=question_id)
    return render(request, 'polls/detail.html', {'question': question})

#### Usign the template for detail

In [ ]:
# Edit polls/templates/polls/detail.html
# At the top show the question and list possible answers for it.
<h1>{{ question.question_text }}</h1>
<ul>
{% for choice in question.choice_set.all %}
    <li>{{ choice.choice_text }}</li>
{% endfor %}
</ul>

### Removing hardcoded URLs in templates


In [ ]:
# Edit polls/templates/polls/index.html
# The url now changes according to changes in polls.urls module
<li><a href="{% url 'detail' question.id %}">{{ question.question_text }}</a></li>

### Namespacing URL names
When django project has alot of apps, it needs to know which app view to create for an url when using {% url %}.

In [ ]:
# Edit polls/urls.py, add app_name = 'polls'
from django.conf.urls import url

from . import views

app_name = 'polls'
urlpatterns = [
    url(r'^$', views.index, name='index'),
    url(r'^(?P<question_id>[0-9]+)/$', views.detail, name='detail'),
    url(r'^(?P<question_id>[0-9]+)/results/$', views.results, name='results'),
    url(r'^(?P<question_id>[0-9]+)/vote/$', views.vote, name='vote'),
]

In [ ]:
# Edit polls/templates/polls/index.html
<li><a href="{% url 'polls:detail' question.id %}">{{ question.question_text }}</a></li>

## Write a simple form


In [ ]:
# Edit polls/templates/polls/detail.html
# It displays a radio botton for each question choice.
<h1>{{ question.question_text }}</h1>

{% if error_message %}<p><strong>{{ error_message }}</strong></p>{% endif %}

<form action="{% url 'polls:vote' question.id %}" method="post">
{% csrf_token %}
{% for choice in question.choice_set.all %}
    <input type="radio" name="choice" id="choice{{ forloop.counter }}" value="{{ choice.id }}" />
    <label for="choice{{ forloop.counter }}">{{ choice.choice_text }}</label><br />
{% endfor %}
<input type="submit" value="Vote" />
</form>

In [ ]:
# Edit polls/views.py
from django.shortcuts import get_object_or_404, render
from django.http import HttpResponseRedirect, HttpResponse
from django.urls import reverse

from .models import Choice, Question
# ...
def vote(request, question_id):
    question = get_object_or_404(Question, pk=question_id)
    try:
        selected_choice = question.choice_set.get(pk=request.POST['choice'])
        # request.POST is a dictionary that lets you access submitted data by key name.
        # request.POSt["choice"] returns the ID of the selected choice as a string.
    except (KeyError, Choice.DoesNotExist):
        # Redisplay the question voting form.
        return render(request, 'polls/detail.html', {
            'question': question,
            'error_message': "You didn't select a choice.",
        })
    else:
        selected_choice.votes += 1
        selected_choice.save()
        # Always return an HttpResponseRedirect after successfully dealing
        # with POST data. This prevents data from being posted twice if a
        # user hits the Back button.
        return HttpResponseRedirect(reverse('polls:results', args=(question.id,)))
    
# and
def results(request, question_id):
    question = get_object_or_404(Question, pk=question_id)
    return render(request, 'polls/results.html', {'question': question})

In [ ]:
# Create polls/results.html template
<h1>{{ question.question_text }}</h1>

<ul>
{% for choice in question.choice_set.all %}
    <li>{{ choice.choice_text }} -- {{ choice.votes }} vote{{ choice.votes|pluralize }}</li>
{% endfor %}
</ul>

<a href="{% url 'polls:detail' question.id %}">Vote again?</a>

## Generic views
detail() and results() views represent a common case of basic Web development: getting data from the database according to a parameter passed in the URL, loading a template and returning the rendered template. Because this is so common, Django provides a shortcut, called the “generic views” system.

In [ ]:
# edit polls/urls.py URLconf
from django.conf.urls import url

from . import views

app_name = 'polls'
urlpatterns = [
    url(r'^$', views.IndexView.as_view(), name='index'),
    url(r'^(?P<pk>[0-9]+)/$', views.DetailView.as_view(), name='detail'),
    url(r'^(?P<pk>[0-9]+)/results/$', views.ResultsView.as_view(), name='results'),
    url(r'^(?P<question_id>[0-9]+)/vote/$', views.vote, name='vote'),
]

In [ ]:
# Remove the old index, detail and results views and use Django's generic views.
from django.shortcuts import get_object_or_404, render
from django.http import HttpResponseRedirect
from django.urls import reverse
from django.views import generic

from .models import Choice, Question


class IndexView(generic.ListView):
    template_name = 'polls/index.html'
    context_object_name = 'latest_question_list'

    def get_queryset(self):
        """Return the last five published questions."""
        return Question.objects.order_by('-pub_date')[:5]


class DetailView(generic.DetailView):
    model = Question
    template_name = 'polls/detail.html'


class ResultsView(generic.DetailView):
    model = Question
    template_name = 'polls/results.html'
# We are using 2 generic views: ListView and DetailView.
# ListView - display a list of objects
# Detail View - deisplay a detail page for a particular type of  object

We are using 2 generic views: ListView and DetailView.
- ListView - display a list of objects
- Detail View - deisplay a detail page for a particular type of  object


- Each generic view needs to know what model it will be acting upon. This is provided using the model attribute.
- The DetailView generic view expects the primary key value captured from the URL to be called "pk", so we’ve changed question_id to pk for the generic views.

# Automated tests
Tests are simple routines that check the operation of your code.

In [ ]:
# question.was_published_recently() returns true, if the question was publishedin the future.
# edit polls/tests.py
import datetime

from django.utils import timezone
from django.test import TestCase

from .models import Question


class QuestionModelTests(TestCase):

    def test_was_published_recently_with_future_question(self):
        """
        was_published_recently() returns False for questions whose pub_date
        is in the future.
        """
        time = timezone.now() + datetime.timedelta(days=30)
        future_question = Question(pub_date=time)
        self.assertIs(future_question.was_published_recently(), False)

What we have done here is created a django.test.TestCase subclass with a method that creates a Question instance with a pub_date in the future. We then check the output of was_published_recently() - which ought to be False.

In [ ]:
# Run the test with
$python manage.py test polls

What happened is this:

- python manage.py test polls looked for tests in the polls application
- it found a subclass of the django.test.TestCase class
- it created a special database for the purpose of testing
- it looked for test methods - ones whose names begin with test
- in test_was_published_recently_with_future_question it created a Question instance whose pub_date field is 30 days - in the future
- … and using the assertIs() method, it discovered that its was_published_recently() returns True, though we wanted it to return False

The test informs us which test failed and even the line on which the failure occurred.

#### Fixing the bug

In [ ]:
# Edit polls/models.py
def was_published_recently(self):
    now = timezone.now()
    return now - datetime.timedelta(days=1) <= self.pub_date <= now

In [ ]:
# Run the test again.
$python manage.py test polls
# It should now be OK.

## More comprehensive tests

In [ ]:
# edit polls/tests.py
polls/tests.py
def test_was_published_recently_with_old_question(self):
    """
    was_published_recently() returns False for questions whose pub_date
    is older than 1 day.
    """
    time = timezone.now() - datetime.timedelta(days=1, seconds=1)
    old_question = Question(pub_date=time)
    self.assertIs(old_question.was_published_recently(), False)

def test_was_published_recently_with_recent_question(self):
    """
    was_published_recently() returns True for questions whose pub_date
    is within the last day.
    """
    time = timezone.now() - datetime.timedelta(hours=23, minutes=59, seconds=59)
    recent_question = Question(pub_date=time)
    self.assertIs(recent_question.was_published_recently(), True)

## Test a view
The polls application is fairly undiscriminating: it will publish any question, including ones whose pub_date field lies in the future. We should improve this. Setting a pub_date in the future should mean that the Question is published at that moment, but invisible until then.

In our first test, we focused closely on the internal behavior of the code. For this test, we want to check its behavior as it would be experienced by a user through a web browser.

### The django test client
Django provides a test Client to simulate a user interacting with the code at the view level. We can use it in tests.py or even in the shell.

We will start again with the shell, where we need to do a couple of things that won’t be necessary in tests.py. The first is to set up the test environment in the shell:

In [ ]:
>>> from django.test.utils import setup_test_environment
>>> setup_test_environment()

setup_test_environment() installs a template renderer which will allow us to examine some additional attributes on responses such as response.context that otherwise wouldn’t be available. Note that this method does not setup a test database, so the following will be run against the existing database and the output may differ slightly depending on what questions you already created.

In [ ]:
>>> from django.test import Client
>>> # create an instance of the client for our use
>>> client = Client()

>>> # get a response from '/'
>>> response = client.get('/')
Not Found: /
>>> # we should expect a 404 from that address; if you instead see an
>>> # "Invalid HTTP_HOST header" error and a 400 response, you probably
>>> # omitted the setup_test_environment() call described earlier.
>>> response.status_code
404
>>> # on the other hand we should expect to find something at '/polls/'
>>> # we'll use 'reverse()' rather than a hardcoded URL
>>> from django.urls import reverse
>>> response = client.get(reverse('polls:index'))
>>> response.status_code
200
>>> response.content
b'\n    <ul>\n    \n        <li><a href="/polls/1/">What&#39;s up?</a></li>\n    \n    </ul>\n\n'
>>> response.context['latest_question_list']
<QuerySet [<Question: What's up?>]>

## Improving our view
The list of polls shows polls that aren’t published yet (i.e. those that have a pub_date in the future). Let’s fix that.

In [ ]:
# edit polls/views.py
from django.utils import timezone

def get_queryset(self):
    """
    Return the last five published questions (not including those set to be
    published in the future).
    """
    return Question.objects.filter(
        pub_date__lte=timezone.now()
    ).order_by('-pub_date')[:5]

Question.objects.filter(pub_date__lte=timezone.now()) returns a queryset containing Questions whose pub_date is less than or equal to - that is, earlier than or equal to - timezone.now.

### Testin out new view

In [ ]:
# edit polls/tests.py
def create_question(question_text, days):
    """
    Create a question with the given `question_text` and published the
    given number of `days` offset to now (negative for questions published
    in the past, positive for questions that have yet to be published).
    """
    time = timezone.now() + datetime.timedelta(days=days)
    return Question.objects.create(question_text=question_text, pub_date=time)


class QuestionIndexViewTests(TestCase):
    def test_no_questions(self):
        """
        If no questions exist, an appropriate message is displayed.
        """
        response = self.client.get(reverse('polls:index'))
        self.assertEqual(response.status_code, 200)
        self.assertContains(response, "No polls are available.")
        self.assertQuerysetEqual(response.context['latest_question_list'], [])

    def test_past_question(self):
        """
        Questions with a pub_date in the past are displayed on the
        index page.
        """
        create_question(question_text="Past question.", days=-30)
        response = self.client.get(reverse('polls:index'))
        self.assertQuerysetEqual(
            response.context['latest_question_list'],
            ['<Question: Past question.>']
        )

    def test_future_question(self):
        """
        Questions with a pub_date in the future aren't displayed on
        the index page.
        """
        create_question(question_text="Future question.", days=30)
        response = self.client.get(reverse('polls:index'))
        self.assertContains(response, "No polls are available.")
        self.assertQuerysetEqual(response.context['latest_question_list'], [])

    def test_future_question_and_past_question(self):
        """
        Even if both past and future questions exist, only past questions
        are displayed.
        """
        create_question(question_text="Past question.", days=-30)
        create_question(question_text="Future question.", days=30)
        response = self.client.get(reverse('polls:index'))
        self.assertQuerysetEqual(
            response.context['latest_question_list'],
            ['<Question: Past question.>']
        )

    def test_two_past_questions(self):
        """
        The questions index page may display multiple questions.
        """
        create_question(question_text="Past question 1.", days=-30)
        create_question(question_text="Past question 2.", days=-5)
        response = self.client.get(reverse('polls:index'))
        self.assertQuerysetEqual(
            response.context['latest_question_list'],
            ['<Question: Past question 2.>', '<Question: Past question 1.>']
        )

### Testing detail views

In [ ]:
# edit polls/views.py
class DetailView(generic.DetailView):
    ...
    def get_queryset(self):
        """
        Excludes any questions that aren't published yet.
        """
        return Question.objects.filter(pub_date__lte=timezone.now())

In [ ]:
# edit polls/tests.py
class QuestionDetailViewTests(TestCase):
    def test_future_question(self):
        """
        The detail view of a question with a pub_date in the future
        returns a 404 not found.
        """
        future_question = create_question(question_text='Future question.', days=5)
        url = reverse('polls:detail', args=(future_question.id,))
        response = self.client.get(url)
        self.assertEqual(response.status_code, 404)

    def test_past_question(self):
        """
        The detail view of a question with a pub_date in the past
        displays the question's text.
        """
        past_question = create_question(question_text='Past Question.', days=-5)
        url = reverse('polls:detail', args=(past_question.id,))
        response = self.client.get(url)
        self.assertContains(response, past_question.question_text)

# Static files

In [1]:
# Add folder polls/static/polls
# Add file style.css
# Add image "background.JPG" to polls/static/polls/images
# Edit style.css
li a {
    color: green;
}

body {
    background: white url("images/background.JPG") no-repeat right bottom;
}

In [ ]:
# Edit polls/templates/polls/index.html
# Add the following code to the top.

{% load static %}

<link rel="stylesheet" type="text/css" href="{% static 'polls/style.css' %}" />